##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# İlk sinir ağınızı eğitin: temel sınıflandırma

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/tr/r1/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Google Colab’da Çalıştır</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/tr/r1/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub'da Kaynağı Görüntüle</a>
  </td>
</table>

Note: Bu dökümanlar TensorFlow gönüllü kullanıcıları tarafından çevirilmiştir.
Topluluk tarafından sağlananan çeviriler gönüllülerin ellerinden geldiğince
güncellendiği için [Resmi İngilizce dökümanlar](https://www.tensorflow.org/?hl=en)
ile bire bir aynı olmasını garantileyemeyiz. Eğer bu tercümeleri iyileştirmek
için önerileriniz var ise lütfen [tensorflow/docs](https://github.com/tensorflow/docs)
havuzuna pull request gönderin. Gönüllü olarak çevirilere katkıda bulunmak için
[docs-tr@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-tr)
listesi ile iletişime geçebilirsiniz.

Bu yardımcı döküman, spor ayakkabısı ve gömlek gibi çeşitli giysi görüntülerini sınıflandırmak için bir sinir ağı modelini eğitir. Örnekte yer alan tüm detayları anlayamadıysanız bu sorun değil, aşağıda ilerledikçe ayrıntıları açıklanacak olan Tensorflow'a hızlı bir genel bakış yapılmaktadır.

Bu örnekte, Tensorflow'da yapay zeka modellerinin oluşturulması ve eğitilmesinde kullanılan yüksek-seviye API olan, [tf.keras](https://www.tensorflow.org/r1/guide/keras) kullanmaktadır.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow ve tf.keras
import tensorflow as tf
from tensorflow import keras

# Yardimci kutuphaneler
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Fashion MNIST veri kümesini yükleyelim

Bu örnek uygulama, 10 kategoride 70,000 siyah-beyaz görüntü içeren [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) veri kümesini kullanmaktadır. Aşağıda görüldüğü gibi bu veri kümesi, çeşitli giyim eşyalarının düşük çüzünürlükteki (28 x 28 piksel) görüntülerini içermektedir :

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figür 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST örnekleri</a> (Zalando tarafından, MIT lisansı ile).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST, klasik [MNIST](http://yann.lecun.com/exdb/mnist/) veri kümesinin yerine kolayca kullanılabilecek şekilde geliştirilmiştir. Klasik MNIST veri kümesi, yukarıda yer alan giysi görüntüleri ile aynı formatta, el yazısı rakam (0, 1, 2, vb) görüntülerini içermektedir.

Fashion MNIST, klasik MNIST'e göre biraz daha zorlayıcı olduğu için ve çeşitliliğin arttırılması amacıyla kullanılmıştır. İki veri kümesi de nispeten küçüktür ve algoritmaların beklendiği gibi çalışıp çalışmadığının doğrulanmasında kullanılırlar. Ayrıca, yazdığımız kodun test edilmesi ve hataların tespit edilmesinde oldukça iyi bir başlangıç noktası oluştururlar.

Oluşturacağımız sinir ağının eğitilmesinde 60,000 görüntü, eğitilmiş sinir ağının görüntü sınıflandırma doğruluğunu değerlendirmek içinse 10,000 görüntü kullanacağız. Fashion MNIST'e TensorFlow içerisinde doğrudan ulaşabilirsiniz, bunun için yapmanız gereken sadece veriyi içeri almak ve yüklemek:

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Veri kümesinin yüklenmesi sonucunda 4 NumPy dizisi oluşur:

* `train_images` ve `train_labels` dizileri *eğitim veri setidir* - modelin eğitilmesinde kullanılır.
* `test_images` ve `test_labels` dizileri *test veri setidir* - modelin test edilmesinde kullanılır.

*train_images, test_images* 28x28 boyutunda ve piksel değerleri 0 ile 255 arasında değişen NumPy dizileridir. *train_labels, test_labels* ise 0 ile 9 arasında değişen ve her biri bir giyim eşyası sınıfı ile eşleşen tam sayı dizisidir:

<table>
  <tr>
    <th>Etiket</th>
    <th>Sınıf</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>Tişört/Üst</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>Pantolon</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>Kazak</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>Elbise</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>Mont</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>Gömlek</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>Spor Ayakkabı</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>Çanta</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>Yarım Bot</td> 
  </tr>
</table>

Veri kümesi içerisindeki her bir görüntü tek bir etiket ile eşleştirilmiştir. *Sınıf isimleri* veri kümesi içerisinde yer almadığı için, daha sonra görüntüleri ekrana yazdırmak için bunları aşağıdaki gibi bir dizi içerisinde saklayalım:

In [0]:
class_names = ['Tişört/Üst', 'Pantolon', 'Kazak', 'Elbise', 'Mont', 
               'Sandal', 'Gömlek', 'Spor Ayakkabı', 'Çanta', 'Yarım Bot']

## Veriyi inceleyelim

Modeli eğitmeye başlamadan önce, veri kümesi yapısını birlikte inceleyelim. Aşağıda, modelin eğitilmesinde kullanılan veri setinin 60,000 görüntüden oluştuğu ve her birinin 28 x 28 piksel olduğunu görmektesiniz:

In [0]:
train_images.shape

Benzer şekilde, eğitim veri setinde 60,000 adet etiket bilgisi yer almaktadır:

In [0]:
len(train_labels)

Her bir etiket 0 ile 9 arasında bir tam sayıdır:

In [0]:
train_labels

Test veri kümesinde 10,000 görüntü mevcuttur. Her bir görüntü, benzer şekilde 28 x 28 piksel den oluşmaktadır:

In [0]:
test_images.shape

Ve test veri seti 10,000 etiket bilgisini kapsamaktadır:

In [0]:
len(test_labels)

## Verileri Ön İşleme

Sinir ağının eğitilmesinden önce verinin bir ön işleme tabi tutulması gerekmektedir. Eğitim veri setindeki ilk görüntüyü inceleyecek olursanız, piksel değerlerinin 0 ile 255 arasında olduğunu göreceksiniz:

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Bu görüntüler ile sinir ağını beslemeden önce, görüntülerin piksel değerlerini 0 ile 1 aralığına ölçekleyeceğiz. Bunun için, piksel değerlerini 255'e böleceğiz. Bu noktada *eğitim veri seti* ile *test veri seti*'nin aynı şekilde ön işlemden geçirilmesi, modelimizin doğru sonuç vermesi açısından önem taşımaktadır:

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

*eğitim veri seti*'nin ilk 25 görüntüsünü, her bir görüntünün altında sınıf etiketi yazacak şekilde ekranda gösterelim. Verinin doğru formatta olduğunu doğruladıktan sonra artık modeli oluşturup eğitmeye hazırız.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Modelin oluşturulması

Sinir ağının oluşturulması için öncelikle model katmanlarının yapılandırılması ve sonrasında modelin derlenmesi gerekmektedir.

### Katmanların hazırlanması

Sinir ağını oluşturan temel yapı taşları *katman*'lardır. Katmanlar, kendilerine beslenen verileri kullanarak bu verilere ait çıkarımlar oluştururlar. Bu çıkarımların, bu örnekte görüntülerin sınıflandırılması olarak karşımıza çıkan problemin çözümüne yardımcı olması beklenir.

Çoğu derin öğrenme modeli, birbirlerine bağlanmış birçok basit katman içermektedir. Çoğu katman, `tf.keras.layers.Dense` gibi, eğitme sürecinde öğrenilen parametrelere sahiptir.

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

Ağımızın ilk katmanı olan `tf.keras.layers.Flatten`, görüntülerin formatını 2 boyutlu sayı dizisinden (28 x 28 piksel),  28 * 28 = 784 piksel değerinden oluşan tek boyutlu bir sayı dizisine çevirir. Bu katmanın, görüntüleri oluşturan piksel satırlarının çıkarılarak, art arda birleştirilmesi ile oluştuğunu düşünebilirsiniz. Bu katmanda öğrenilecek parametre olmayıp, sadece görüntünün formatını düzenler.

Görüntüyü oluşturan pikselleri tek boyutlu sayı dizisine düzleştirdikten sonra, ağımız ardaşık iki `tf.keras.layers.Dense` katmanını içerir. Bunlara, yoğun-bağlı veya tam-bağlı ağ katmanları denir.  İlk 'Yoğun' katman 128 neron'a (düğüm) sahiptir. İkinci katman is 10 neronlu 'softmax' katmanıdır. Bu son katmanın çıktısı, toplam değerleri 1' eşit olan ve 10 farklı olasılık sonucunu içeren sayı dizisidir. Her bir düğüm, mevcut görüntünün hangi sınıfa ait olduğu belirten olasılık değerini içerir.

### Modelin derlenmesi

Modelin eğitilmeye tamamıyla hazır olması öncesinde bir kaç düzenleme daha yapılması gerekmektedir. Bu düzenlemeler modelin 'derlenme' adımında eklenmektedir:

* *Kayıp Fonksiyonu - Loss Function* — Bu fonksiyon modelin eğitim sürecinde ne kadar doğru sonuç verdiğini ölçer. Bu fonksiyonun değerini en aza indirgeyerek, modelin doğru istikamete "yönlendirmek" isteriz.
* *Eniyileme - Optimizer* — Beslenen veriler ve kayıp fonksiyonu ile modelin nasıl güncellediğini belirler
* *Metrikler - Metrics* — Eğitim ve test adımlarını gözlemlemek için kullanılır. Aşağıdaki örnekte, *doğruluk-accuracy*, modelin doğru sınıfladığı görüntü oranı, kullanılmaktadır.

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Modelin eğitilmesi

Sinir ağının eğitilmesi aşağıdaki adımları gerektirir:

1. Eğitim veri setinin modele beslenmesi - bu örnekte veri setimiz yukarıda açıklanan 'train_images' ve 'train_labels' dizileridir.
2. Model etiketleri ve görüntüleri kullanarak çıkarımlar yapar ve öğrenir.
3. Modelden test veri setini - bu örnekte 'test_images' dizisidir, kullanarak tahminleme yapmasını isteriz. Sonucu 'test_labels' dizisindeki etiket ile eşleştirerek, bu kestirimlerin doğruluğunu teyid edebiliriz.

Eğitimi başlatmak için 'model.fit' methodu çalıştırılır:

In [0]:
model.fit(train_images, train_labels, epochs=5)

Model eğitimi süresince, kayıp ve doğruluk metrikleri ekranda gösterilir. Örneğimizdeki model, eğitim veri setiyle 0.88 (or 88%) doğruluk eğerine ulaşır.

## Model doğruluğunun değerlendirlmesi

Sonrasında, modelin test veri seti ile nasıl bir performans gösterdiğini karşılaştıralım:

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('Test accuracy:', test_acc)

Test veri seti ile aldığımız model doğruluk değerinin, eğitim veri seti ile aldığımız model doğruluk değerinden biraz daha düşük olduğunu görmekteyiz. Eğitim doğruluk değeri ile test doğruluk değeri arasındaki bu farka *aşırı uyum-overfitting* denir. Aşırı uyum, modelin yeni veriler ile tahminleme yaparken, eğitim veri setine göre daha kötü performans göstermesi durumudur. 

## Modeli kullanarak tahminleme yapalım

Eğitilmiş modelimizi kullanarak, bir kaç görüntü için tahminleme yapabiliriz:

In [0]:
predictions = model.predict(test_images)

Aşağıda, test veri setinde yer alan her bir görüntü için, modelimiz etiket sınıflandırması yapmaktadır. İlk tahminlemeye birlikte bakalım:

In [0]:
predictions[0]

Tahminleme sonucu, 10 sayıdan oluşan bir dizi elde ederiz. Bu sayı dizisi bize, görüntünün 10 farklı sınıftan hangi giysi türüne ait olduğuna dair modelin "güvenini" tanımlamaktadır. Bu değerlere bakarak, hangi etiket sınıfının en yüksek güven değerine sahip olduğunu görebiliriz:

In [0]:
np.argmax(predictions[0])

Modelimiz yarım bot etiketi, (`veya class_names[9]`) için en yüksek kestirim güven değeri vermektedir. Ve test veri setindeki etikete bakarak sonucun doğru olduğunu görebiliriz:

In [0]:
test_labels[0]

10 farklı sınıfın tümüne bakabilmek için sonucun grafiğini oluşturabiliriz:

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

0'ıncı görüntüye, tahminlere ve tahmin dizisine bakalım: 

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

Çeşitli görüntüleri, tahminlemeleri ile ekranda gösterelim. Doğru tahminleme sonuçları mavi, yanlış olanlar ise kırmızı ile ekranda gösterilecektir. Rakamlar ise, 100 üzerinden, yapılan tahminlemenin güven değerini vermektedir. Güven değeri yüksek olsa bile, sonucun yanlış olabileceğini görebilirsiniz. 

In [0]:
# Ilk X resmi, tahmin edilen etiketini ve asil etiketlerini cizelim.
# Dogru tahminler mavi, yanlis tahminler ise kirmizi renktedir.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

Son olarak, eğitilmiş modeli kullanarak tek bir görüntü üzerinden tahminleme yapalım: 

In [0]:
# Test veri setinden bir resim secelim.
img = test_images[0]

print(img.shape)

`tf.keras` modelleri, *veri yığınları* içerisindeki örnekler üzerinden tahminleme yapmak üzere optimize edilmiştirler. Tek bir görüntü kullanmamıza rağmen, bu nedenle görüntüyü bir listeye aktarmamız gerekmektedir:

In [0]:
# Resmi tek ogesi kendisi olacagi bir listeye aktaralim.
img = (np.expand_dims(img,0))

print(img.shape)

Şimdi görüntüyü tahminleyelim:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(0, predictions_single, test_labels)
plt.xticks(range(10), class_names, rotation=45)
plt.show()

`model.predict` çalıştırıldığında, veri yığını içerisindeki her bir görüntüye ait bir liste verir. Yığın içerisinden görüntümüze (örneğimizdeki tek görüntü) ait tahminleme sonuçlarını alalım:

In [0]:
prediction_result = np.argmax(predictions_single[0])
print(prediction_result)

Daha önceden olduğu gibi, modelimiz etiket değeri olarak 9'u vermektedir.